In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.sparse.linalg
from scipy.sparse import csr_matrix, lil_matrix

# Problema: A, T y W

In [2]:
T = np.zeros((15,3))
T[12,0] = 1
T[13,1] = 1
T[14,2] = 1
W = np.zeros((15,21))
for i in range(0,9):
    W[i,i] = 1
    W[i,i+9] = 1
    W[9+i%3,i] = 1
W[14,20]= -1
W[13,19]= -1
W[12,18] = -1

#plt.imshow(T, cmap='Greens')
#plt.gca().axes.get_yaxis().set_visible(False)
#plt.gca().axes.get_xaxis().set_visible(False)
#plt.imshow(W, cmap='PRGn')
#plt.gca().axes.get_yaxis().set_visible(False)
#plt.gca().axes.get_xaxis().set_visible(False)

## Gurobi

In [4]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum

In [5]:
model = gp.Model('Intento')
y = model.addVars([1,2], lb = 0, vtype = GRB.CONTINUOUS)
obj = np.array([-24,-28])
model.modelSense = GRB.MINIMIZE
model.setObjective( quicksum(y[a]*obj[a-1] for a in range(1,3)) )

lhs_ineq = np.array([[6,10],[8,5],[1,0],[0,1]])
rhs_ineq = np.array([2400,1600,500,100])

model.addMConstr (lhs_ineq, x = None, sense='<=', b = rhs_ineq, name="cons" )
model.optimize()
print("\nResultado:\n")
print(y,"\n")
print("Pi del modelo: \n")
print(model.pi)

Academic license - for non-commercial use only - expires 2021-06-26
Using license file C:\Users\melir\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xb42b4eb3
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -6.1000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective -6.100000000e+03

Resultado:

{1: <gurobi.Var C0 (value 137.5)>, 2: <gurobi.Var C1 (value 100.0)>} 

Pi del modelo: 

[0.0, -3.0, 0.0, -13.0]


In [42]:
# D es una matriz rx2 donde en la primera columna están los valores de D_l y en la segunda los de d_l
def problemaMaestro(D, E, c, A, b):
    model = gp.Model('Maestro')
    
    #Verifica que las entradas estén bien.
    if( D.any() and D.shape[1] != c.size+1 ):
        return('Error en las entradas. D debe ser r x (c.size+1)')
    
    if( E.any() and E.shape[1] != c.size+1 ):
        return('Error en las entradas. E debe ser s x (c.size+1)')
    
    if( E.any() == False and E.any() == False ): #Está funcionando.
        x = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS)
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(x[a]*c[a] for a in range(0,c.size)) )
        
        model.addMConstr (A, x = None, sense='=', b = b, name="constr" )
        model.optimize()
        rta = np.array([x[i].x for i in range(0,c.size)])
        return(rta,model.pi)
    
    elif( D.any() == False ): #Está funcionando.
        e = E[:,-1]
        E = np.hstack((E[:,:-1], np.ones((E.shape[0],1))))
        varx = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        theta = model.addVars(1, lb = float('-Inf'), vtype = GRB.CONTINUOUS, name = 'theta')

        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(varx[a]*c[a] for a in range(0,c.size)) + theta[0] )

        list = [varx[i] for i in range(0,c.size)]
        model.addMConstr (A, x = list, sense='=', b = b, name="constr" )

        list.append(theta[0])
        model.addMConstr (E, x = list, sense='>=', b = e, name="constrE" ) 

        model.optimize()
        rta = np.array([varx[i].x for i in range(0,c.size)])
        rta = np.concatenate((rta,np.array([theta[0].x])))
        return(rta,model.pi)
    
    elif( E.any() == False ):
        d = D[:,-1]
        D = D[:,:-1] 

        x = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS)
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(x[a]*c[a] for a in range(0,c.size)) )
        
        model.addMConstr (A, x = None, sense='=', b = b, name="constr" )
        
        model.addMConstr (D, x = None, sense='>=', b = d )
        
        model.optimize()
        rta = np.array([x[i].x for i in range(0,c.size)])
        return(rta,model.pi)
    
    else: 
        e = E[:,-1]
        E = np.hstack((E[:,:-1], np.ones((E.shape[0],1))))
        d = D[:,-1]
        D = D[:,:-1] 
        
        varx = model.addVars(range(0,c.size), lb = 0, vtype = GRB.CONTINUOUS, name='varx')
        theta = model.addVars(1, lb = float('-Inf'), vtype = GRB.CONTINUOUS, name = 'theta')
        model.modelSense = GRB.MINIMIZE
        model.setObjective( quicksum(varx[a]*c[a] for a in range(0,c.size)) + theta[0] )
        list = [varx[i] for i in range(0,c.size)]
        model.addMConstr (A, x = list, sense='=', b = b, name="constr" )
        model.addMConstr (D, x = list, sense='>=', b = d, name="constrD" )

        list.append(theta[0])
        model.addMConstr (E, x = list, sense='>=', b = e, name="constrE" ) 
        
        model.optimize()
        rta = np.array([varx[i].x for i in range(0,c.size)])
        rta = np.concatenate((rta,np.array([theta[0].x])))        
        return(rta,model.pi)
            


In [43]:
#Ejemplo caso 2:

E = np.array([[83.52, 180.48, 0,0,0,-520]])

c = np.array([100,150,0,0,0])
A = np.array([[1,1,1,0,0],[1,0,0,-1,0],[0,1,0,0,-1]])
b = np.array([120,40,20])


rta = problemaMaestro(np.array([]), E, c, A, b)
print(rta)



Caso 2
[[ 83.52 180.48   0.     0.     0.     1.  ]]
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 6 columns and 10 nonzeros
Model fingerprint: 0x81ebe631
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+02]
Presolve removed 4 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.2992000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -2.299200000e+03
[40. 80.  0.  0. 60.]
[-18299.2]
(array([    40. ,     80. ,      0. ,      0. ,     60. , -18299.2]), [-30.47999999999999, 46.959999999999994, 0.0, 1.0])


## Escenarios
Creamos 5000 escenarios para el vector $\zeta = (r_1,r_2,r_3,d_1,d_2,d_3)$ de forma uniforme. Pueden tomar valores enteros, $ 20 \leq r_1,r_2,r_3 \leq 30$ y $(d_1,d_2,d_3) \in \{(0,8,25),(1,15,35),(1,11,40),(3,20,55),(4,20,70)\}$. Tenemos entonces $10\times 10\times 10 \times 5 = 5000$ valores posibles para $\zeta$. Cada uno lo tomamos con probabilidad $p = \frac{1}{5000}$.

In [ ]:
obj = np.array([10,15,20,float('-Inf')])
print(obj)
print(range(1,obj.size+1))
